# Levenshtein Algorithm: An optimization walkthrough

In [2]:
%load_ext Cython

In [3]:
import random
import string

alphanum = string.lowercase + string.uppercase + string.digits
s1 = ''.join(random.choice(alphanum) for _ in xrange(1024))
s2 = ''.join(random.choice(alphanum) for _ in xrange(1024))

In [3]:
def levenshtein_1_py(s1, s2):

    n1 = len(s1) + 1
    n2 = len(s2) + 1

    m = [[0 for _ in range(n2)] for _ in range(n1)]

    for i in range(0, n1):
        m[i][0] = i

    for j in range(0, n2):
        m[0][j] = j

    for j in range(1, n2):
        for i in range(1, n1):
            if s1[i-1] == s2[j-1]:
                m[i][j] = m[i-1][j-1]
            else:
                m[i][j] = min(m[i-1][j] + 1, m[i][j-1] + 1, m[i-1][j-1] + 1)

    return m[n1-1][n2-1]

In [4]:
%%cython --annotate
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
def levenshtein_1(s1, s2):

    cdef n1 = len(s1) + 1
    cdef n2 = len(s2) + 1

    m = [[0 for _ in range(n2)] for _ in range(n1)]

    for i from 0 <= i < n1:
        m[i][0] = i

    for j from 0 <= j < n2:
        m[0][j] = j

    for j from 1 <= j < n2:
        for i from 1 <= i < n1:
            if s1[i-1] == s2[j-1]:
                m[i][j] = m[i-1][j-1]
            else:
                m[i][j] = min(m[i-1][j] + 1, m[i][j-1] + 1, m[i-1][j-1] + 1)

    return m[n1-1][n2-1]

In [5]:
def levenshtein_2_py(s1, s2):

    n1 = len(s1)
    n2 = len(s2)
    
    column = range(n1 + 1)

    for j in range(1, n2 + 1):
        column[0] = j
        last_diag = j - 1
        for i in range(1, n1 + 1):
            current = column[i]
            cost = 0 if s1[i-1] == s2[j-1] else 1
            column[i] = min(column[i] + 1, column[i-1] + 1, last_diag + cost)
            last_diag = current
            
    return column[n1]

In [6]:
%%cython --annotate
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
def levenshtein_2(s1, s2):

    cdef n1 = len(s1)
    cdef n2 = len(s2)
    
    column = range(n1 + 1)

    for j in range(1, n2 + 1):
        column[0] = j
        last_diag = j - 1
        for i in range(1, n1 + 1):
            current = column[i]
            cost = 0 if s1[i-1] == s2[j-1] else 1
            column[i] = min(column[i] + 1, column[i-1] + 1, last_diag + cost)
            last_diag = current
    
    return column[n1]

In [7]:
%%cython --annotate
import cython

import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def levenshtein_3(const char* s1, const char* s2):

    cdef int n1 = len(s1)
    cdef int n2 = len(s2)
    cdef int current, last_diag, cost
    cdef int i, j
    
    column = range(n1 + 1)
    
    for j in range(1, n2 + 1):
        column[0] = j
        last_diag = j - 1
        for i in range(1, n1 + 1):
            current = column[i]
            cost = 0 if s1[i-1] == s2[j-1] else 1
            column[i] = min(column[i] + 1, column[i-1] + 1, last_diag + cost)
            last_diag = current
            
    return column[n1]

In [8]:
%%cython --annotate
import cython

import numpy as np
cimport numpy as np

np.import_array()

@cython.boundscheck(False)
@cython.wraparound(False)
def levenshtein_4(const char* s1, const char* s2):

    cdef int n1 = len(s1)
    cdef int n2 = len(s2)
    cdef int last_diag, cost, current
    
    cdef Py_ssize_t i, j
    
    cdef Py_ssize_t m1 = n1 + 1
    cdef Py_ssize_t m2 = n2 + 1

    cdef np.ndarray[np.int_t, ndim=1] column = np.zeros(m1, dtype=np.int)
    
    for i in range(m1):
        column[i] = i

    for j in range(1, m2):
        column[0] = j
        last_diag = j - 1
        for i in range(1, m1):
            current = column[i]
            cost = 0 if s1[i-1] == s2[j-1] else 1
            column[i] = min(column[i] + 1, column[i-1] + 1, last_diag + cost)
            last_diag = current
            
    return column[n1]

In [29]:
%%cython --annotate
import cython

import numpy as np
cimport numpy as np

np.import_array()

@cython.boundscheck(False)
@cython.wraparound(False)
def levenshtein_5(const char* s1, const char* s2):

    cdef int n1 = len(s1)
    cdef int n2 = len(s2)
    cdef int last_diag, cost, current
    
    cdef Py_ssize_t i, j
    
    cdef Py_ssize_t m1 = n1 + 1
    cdef Py_ssize_t m2 = n2 + 1
    
    cdef np.ndarray[np.npy_int, ndim=1] column = np.PyArray_ZEROS(1, <np.npy_intp*> &m1, np.NPY_INT, 0)
    
    for i in range(m1):
        column[i] = i

    for j in range(1, m2):
        column[0] = j
        last_diag = j - 1
        for i in range(1, m1):
            current = column[i]
            cost = 0 if s1[i-1] == s2[j-1] else 1
            column[i] = min(column[i] + 1, column[i-1] + 1, last_diag + cost)
            last_diag = current
            
    return column[n1]

In [10]:
%%cython --annotate
import cython

from libc.stdlib cimport abort, malloc, free


@cython.boundscheck(False)
@cython.wraparound(False)
def levenshtein_6(const char* s1, const char* s2):

    cdef int n1 = len(s1)
    cdef int n2 = len(s2)
    cdef int last_diag, cost, current
    
    cdef Py_ssize_t i, j
    
    cdef Py_ssize_t m1 = n1 + 1
    cdef Py_ssize_t m2 = n2 + 1
    
    cdef int* column = <int*> malloc(m1 * sizeof(int))
    if not column:
        abort()
    
    for i in range(m1):
        column[i] = i

    for j in range(1, m2):
        column[0] = j
        last_diag = j - 1
        for i in range(1, m1):
            current = column[i]
            cost = 0 if s1[i-1] == s2[j-1] else 1
            column[i] = min(column[i] + 1, column[i-1] + 1, last_diag + cost)
            last_diag = current
    
    current = column[n1]
    free(column)
    return current

In [11]:
%%cython --annotate
import cython

from libc.stdlib cimport abort, malloc, free

@cython.boundscheck(False)
@cython.wraparound(False)
cdef int cy_levenshtein(const char* s1, int n1, const char* s2, int n2) nogil:
    cdef int last_diag, cost, current
    cdef Py_ssize_t i, j
    cdef Py_ssize_t m1 = n1 + 1
    cdef Py_ssize_t m2 = n2 + 1
    
    if not n1:
        return n2
    if not n2:
        return n1
    
    cdef int* column = <int*> malloc(m1 * sizeof(int))
    if not column:
        abort()
    
    for i in range(m1):
        column[i] = i

    for j in range(1, m2):
        column[0] = j
        last_diag = j - 1
        for i in range(1, m1):
            current = column[i]
            cost = 0 if s1[i-1] == s2[j-1] else 1
            column[i] = min(column[i] + 1, column[i-1] + 1, last_diag + cost)
            last_diag = current
            
    current = column[n1]
    free(column)
            
    return current

def levenshtein_7(s1, s2):
    return cy_levenshtein(s1, len(s1), s2, len(s2))

In [60]:
%%cython --annotate -f -c-fopenmp --link-args=-fopenmp -c-g
# cython: cdivision=True
# cython: boundscheck=False
# cython: wraparound=False

from libc.stdlib cimport abort, malloc, free

import numpy as np
cimport numpy as np

np.import_array()

from cython.parallel import parallel, prange

cdef int cy_levenshtein(const Py_UNICODE* s1, int n1, const Py_UNICODE* s2, int n2) nogil:
    cdef int last_diag, cost, current
    cdef Py_ssize_t i, j
    cdef Py_ssize_t m1 = n1 + 1
    cdef Py_ssize_t m2 = n2 + 1
    
    if not n1:
        return n2
    if not n2:
        return n1
    
    cdef int* column = <int*> malloc(m1 * sizeof(int))
    if not column:
        abort()
    
    for i in range(m1):
        column[i] = i

    for j in range(1, m2):
        column[0] = j
        last_diag = j - 1
        for i in range(1, m1):
            current = column[i]
            cost = 0 if s1[i-1] == s2[j-1] else 1
            column[i] = min(column[i] + 1, column[i-1] + 1, last_diag + cost)
            last_diag = current
            
    current = column[n1]
    free(column)
            
    return current

def levenshtein_8(s1, s2):
    return cy_levenshtein(s1, len(s1), s2, len(s2))


def levenshtein_8_pdist(values):
    # condensed distance representation
    cdef int n = len(values)
    cdef int m = n * (n - 1) // 2
    cdef np.ndarray[np.int_t, ndim=1] result = np.zeros(m, dtype=np.int)
    
    cdef Py_ssize_t i, j
    cdef int offset

    for i in range(n):
        offset = i * n - (i + 1) * (i + 2) // 2
        for j in range(i + 1, n):
            result[offset + j] = cy_levenshtein(values[i], len(values[i]), values[j], len(values[j]))
    
    return result


def levenshtein_8_pdist_parallel(values):
    # condensed distance representation
    cdef int n = len(values)
    cdef int m = n * (n - 1) // 2
    cdef np.ndarray[np.int_t, ndim=1] result = np.zeros(m, dtype=np.int)
    
    cdef Py_ssize_t i, j
    cdef int offset
    
    cdef int* lengths = <int*> malloc(n * sizeof(int))        
    cdef Py_UNICODE** strings = <Py_UNICODE**> malloc(n * sizeof(Py_UNICODE*))

    for i in range(n):
        strings[i] = values[i]
        lengths[i] = len(strings[i])

    with nogil:
        for i in range(n):
            offset = i * n - (i + 1) * (i + 2) // 2
            for j in prange(i + 1, n, schedule='guided'):
                result[offset + j] = cy_levenshtein(strings[i], lengths[i], strings[j], lengths[j])

    free(lengths)
    free(strings)
    
    return result

In [ ]:
%%cython --annotate -f -c-fopenmp --link-args=-fopenmp -c-g
# cython: cdivision=True
# cython: boundscheck=False
# cython: wraparound=False

from libc.stdlib cimport abort, malloc, free

import numpy as np
cimport numpy as np

np.import_array()

from cython.parallel import parallel, prange

cdef void cy_levenshtein(const Py_UNICODE* s1, int n1,
                        const Py_UNICODE* s2, int n2,
                        np.int_t[:] column) nogil:
    cdef int last_diag, cost, current
    cdef Py_ssize_t i, j
    cdef Py_ssize_t m1 = n1 + 1
    cdef Py_ssize_t m2 = n2 + 1
    
    for i in range(m1):
        column[i] = i

    for j in range(1, m2):
        column[0] = j
        last_diag = j - 1
        for i in range(1, m1):
            current = column[i]
            cost = 0 if s1[i-1] == s2[j-1] else 1
            column[i] = min(column[i] + 1, column[i-1] + 1, last_diag + cost)
            last_diag = current


def levenshtein_9(s1, s2):
    cdef int n1 = len(s1)
    cdef int n2 = len(s2)
    
    cdef np.npy_intp dim = n1
    cdef np.ndarray[np.npy_long, ndim=1] column = np.PyArray_ZEROS(1, &dim, np.NPY_LONG, 0)
    
    cy_levenshtein(s1, n1, s2, n2, column)
    
    return column[n1]


def levenshtein_9_pdist(values):
    # condensed distance representation
    cdef int n = len(values)
    cdef int m = n * (n - 1) // 2

    cdef np.npy_intp dim = m
    cdef np.ndarray[np.npy_long, ndim=1] result = np.PyArray_ZEROS(1, &dim, np.NPY_LONG, 0)

    cdef Py_ssize_t i, j
    cdef int offset

    for i in range(n):
        offset = i * n - (i + 1) * (i + 2) // 2
        for j in range(i + 1, n):
            result[offset + j] = levenshtein_9(values[i], values[j])
    
    return result


def levenshtein_9_pdist_parallel(values):
    # condensed distance representation
    cdef int n = len(values)
    cdef int m = n * (n - 1) // 2
    
    cdef np.npy_intp dim = m
    cdef np.ndarray[np.npy_long, ndim=1] result = np.PyArray_ZEROS(1, &dim, np.NPY_LONG, 0)

    cdef Py_ssize_t i, j, k
    cdef int offset
    
    cdef int* lengths = <int*> malloc(n * sizeof(int))        
    cdef Py_UNICODE** strings = <Py_UNICODE**> malloc(n * sizeof(Py_UNICODE*))
    
    cdef np.ndarray[np.npy_long, ndim=1] column

    for i in range(n):
        strings[i] = values[i]
        lengths[i] = len(strings[i])

    with nogil, parallel():
        for i in range(n):
            offset = i * n - (i + 1) * (i + 2) // 2
            for j in prange(i + 1, n, schedule='guided'):
                with gil:
                    column = np.PyArray_ZEROS(1, <np.npy_intp*> &(lengths[i]), np.NPY_LONG)
                cy_levenshtein(strings[i], lengths[i], strings[j], lengths[j], column)
                result[offset + j] = column[lengths[i]]

    free(lengths)
    free(strings)
    
    return result

In [30]:
%timeit -r5 levenshtein_1_py(s1, s2)

1 loops, best of 5: 751 ms per loop


In [31]:
%timeit -r5 levenshtein_2_py(s1, s2)

1 loops, best of 5: 515 ms per loop


In [32]:
%timeit -r5 levenshtein_1(s1, s2)

1 loops, best of 5: 532 ms per loop


In [33]:
%timeit -r5 levenshtein_2(s1, s2)

1 loops, best of 5: 245 ms per loop


In [34]:
%timeit -r5 levenshtein_3(s1, s2)

10 loops, best of 5: 84.5 ms per loop


In [45]:
%timeit -r5 levenshtein_4(s1, s2)

100 loops, best of 5: 4.28 ms per loop


In [50]:
%timeit -r5 levenshtein_5(s1, s2)

100 loops, best of 5: 3.79 ms per loop


In [49]:
%timeit -r5 levenshtein_6(s1, s2)

100 loops, best of 5: 4.63 ms per loop


In [48]:
%timeit -r5 levenshtein_7(s1, s2)

100 loops, best of 5: 4.63 ms per loop


In [61]:
%timeit -r5 levenshtein_8(unicode(s1), unicode(s2))

100 loops, best of 5: 4.67 ms per loop


In [11]:
%timeit -r5 levenshtein_9(unicode(s1), unicode(s2))

100 loops, best of 5: 4.32 ms per loop


In [52]:
%time
test_data = [
    ("kitten", "sitting", 3),
    ("kitten", "kitten", 0),
    ("", "", 0),
    ("meilenstein", "levenshtein", 4),
    ("levenshtein", "frankenstein", 6),
    ("confide", "deceit", 6),
    ("CUNsperrICY", "conspiracy", 8),
]
test_functions = [
    levenshtein_1_py,
    levenshtein_2_py,
    levenshtein_1,
    levenshtein_2,
    levenshtein_3,
    levenshtein_4,
    levenshtein_5,
    levenshtein_6,
    levenshtein_7,
]
def test():
    for func in test_functions:
        for s1, s2, dist in test_data:
            assert func(s1, s2) == dist, "%s(%s, %s) != %s" % (func.__name__, s1, s2, dist)
            
test()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.96 µs


In [54]:
%time
assert levenshtein_8(u"a", u"b\xf1") == 2
def test():
    for s1, s2, dist in test_data:
        assert levenshtein_8(unicode(s1), unicode(s2)) == dist, (s1, s2, dist)
test()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


In [15]:
words = !shuf -n1000 /usr/share/dict/words
words = [s.decode('utf-8') for s in words]

In [14]:
from scipy.spatial.distance import squareform

out = levenshtein_8_pdist_parallel(words)
assert np.allclose(levenshtein_8_pdist(words), out)
squareform(out)

NameError: name 'levenshtein_8_pdist_parallel' is not defined

In [62]:
%timeit -r5 levenshtein_8_pdist(words)

1 loops, best of 5: 179 ms per loop


In [65]:
%timeit -r5 levenshtein_8_pdist_parallel(words)

10 loops, best of 5: 87.1 ms per loop


In [17]:
%timeit -r5 levenshtein_9_pdist(words)

1 loops, best of 5: 831 ms per loop
